# Using MCP Servers with Strands Agents

This notebook demonstrates how to integrate Model Context Protocol (MCP) servers as tools for Strands Agents. MCP provides a standardized way to extend AI agents with external tools and data sources, enabling more powerful and specialized capabilities beyond basic language model interactions. By connecting MCP servers to your agents, you can access file systems, databases, APIs, and custom business logic seamlessly.

**Why use MCP servers?** They eliminate the need to rebuild common functionality, provide secure sandboxed tool execution, and enable modular agent architectures that can be easily extended and maintained.

**Key takeaways:** You'll learn to configure existing MCP servers for immediate productivity gains, build custom servers tailored to your specific use cases, and understand best practices for local development and deployment of MCP-enabled agent systems.

### Prerequsites

In [ ]:
# Install all required packages from requirements.txt
!pip install -r requirements.txt --quiet

In [ ]:
# Import required libraries
import os, time, boto3, json

from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import use_aws, file_write, journal, file_read, file_write, sleep, python_repl, retrieve, current_time
from strands.telemetry import StrandsTelemetry

from datetime import datetime
from pprint import pprint

# Bypass tool consent for automated execution
os.environ["BYPASS_TOOL_CONSENT"] = "true"
os.environ["PYTHON_REPL_INTERACTIVE"] = "False"


In [ ]:
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%m/%d %H:%M:%S',
    filename='strands_debug.log'
)

### 1. A simple agent without any tool
##### Let's see how a simple Strands Agent works
![Write code](just_the_ai_agent.png)

In [ ]:
# Give a task to the agent
task = f"""
Write Python code to find differentiation of a quadratic equation.
"""

# Defne a model. Initialize Claude 3.7 Sonnet model via Bedrock
model = BedrockModel(model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0", temperature=0.1)

# Define an agent. Initialize a Strands Agent with the model
agent = Agent(model=model)

# Give the task to the agent
response = agent(task)

###  2. Let's add built-in Strands Agent tool
*Let's execute the python code using a built-in tool*

Now, we will give *tools* to the agent. Tools can be provided as Python libraries. Strands Agents provides built-in tools as a part of the open-source project.
Remember, these tools are functions running inside the same compute engine as the agent itself.

![Execute code](ai_agent_tool.png)

In [ ]:
# Give a task to the agent
task = f"""
Write Python code to find differentiation of a qudratic equation using Numpy.
Print the results for this equation y = 20*(x**3) + 30*(x**2) - 7.
"""

# Defne a model. Initialize Claude 3.7 Sonnet model via Bedrock
model = BedrockModel(model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0", temperature=0.1)

# Define an agent. Initialize a Strands Agent with the model,
# python_repl tool helps execute the python code
# file_write tool helps save the code to a python file
agent = Agent(model=model, tools=[python_repl, file_write])

# Give the task to the agent
response = agent(task)

###  3. Adding an MCP client as a tool for a remote server

Now, we will extend the scope of tools to remote services.
<br/>
Following cell shows an example that requires access to AWS documents. AWS Lab provides an MCP server which provides a knowledge base for AWS documentations. <br/>
By using this, Stands Agents can retrieve documents without implementing its own knowledge base.

In [ ]:
# Ask a question that is very specific to your corporate product or process.
# The agent will not be able to answer the question, assuming it has never seen that data when it was trained.
from IPython.display import display, Markdown
answer = agent("I want to learn about Amazon Bedrock AgentCore which was announced in July 2025."
      "Explain the difference between Bedrock AgentCore short-term memory and long-term memory").message["content"][0]["text"]
display(Markdown(answer))

#### Step 1: Create an MCP Client

In [ ]:
#Let's create a reusable function to create an MCPP Client

# Import MCP client libraries
from mcp import stdio_client, StdioServerParameters # Core MCP protocol components
from strands.tools.mcp import MCPClient # Strands wrapper for MCP functionality

# Typical usage of the following function is:
# aws_doc_client = create_mcp_client("uvx", ["awslabs.aws-documentation-mcp-server@latest"])
# uvx is a python package runner. 
# awslabs.aws-documentation-mcp-server@latest is an MCP server

def create_mcp_client(command, args, env={"AWS_REGION": "us-west-2","FASTMCP_LOG_LEVEL": "ERROR"}):
    return MCPClient(lambda: stdio_client( # The MCP client created connects to the MSCP server via stdin/stdout
        StdioServerParameters(  # Configures how to launch the MCP server process
            command= command, # The executable to run such as uvx, which is a python package runner
            args= args, # One of the argument can be the package name of MCP server that needs to be downloaded and run as a sub process
            env= env
        )
    ))

##### Let's use an MCP Server awslabs.aws-documentation-mcp-server@latest
It's a documentation mcp server that will search and answer AWS documentation related questions
![Tools from Documentation MCP Server](tools_from_doc_MCP_server2.png)

In [ ]:
# Let's first print the tools that the MCP server offers and its description
# Keep in mind that the LLM uses the tool description to determine if it's the right tool for a task.

#create the client
aws_doc_client = create_mcp_client("uvx", ["awslabs.aws-documentation-mcp-server@latest"])

with aws_doc_client:
    # Get the tools that the mcp client has to offer
    doc_tools = aws_doc_client.list_tools_sync()

    for tool in doc_tools:
        print(f"Tool: {tool.tool_name}")
        if hasattr(tool, 'tool_spec'):            
            print(f"Tool: {tool.tool_spec['description']}")
            print(json.dumps(tool.tool_spec, indent=2)) # uncomment this to see function parameters and what they mean               
#       print("-" * 50)

#### Step 2: Let the agent to use the MCP client as a tool 

In [ ]:
# Let us ask the same question to an agent using MCP as a tool
task = """I want to learn about Amazon Bedrock AgentCore which was announced in July 2025.
       Explain the difference between Bedrock AgentCore short-term memory and long-term memory"""

#create the client
aws_doc_client = create_mcp_client("uvx", ["awslabs.aws-documentation-mcp-server@latest"])

#create the model
model = BedrockModel(model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", temperature=0.1)
# model = BedrockModel(model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0", temperature=0.1)
# model = BedrockModel(model_id="us.deepseek.r1-v1:0", temperature=0.1)
# model = BedrockModel(model_id="openai.gpt-oss-120b-1:0", temperature=0.1)

with aws_doc_client:
    # Get the tools that the mcp client has to offer
    doc_tools = aws_doc_client.list_tools_sync()

    #create the agent and pass the tools recieved from the MCP client
    aws_doc_agent = Agent(model=model, tools=[doc_tools])
    answer = aws_doc_agent(task).message["content"][0]["text"]

from IPython.display import display, Markdown
display(Markdown(answer))

### 4. Let's use multiple MCP Servers and a tool in the same Agent
![Multiple MCP Servers](multiple_mcp_servers.png)

In [ ]:
# Use multiple MCP servers and a tool to research Amazon SgaeMaker.
# Store the research in a file and pricing info in a dynamo db table.
task = """
- What are the features and benefits of Amazon Sagemaker's fine tuning ability of large language models. Store the various features and benefits in dictionary format in a Dynamo DB table named my_ddb_for_sagemaker
- How can I fine tune LLama 8B model using SageMaker?
- What type of SageMaker instances would I need to fine tune Llama 8B model?
- How much do they cost?
- Store the instance - cost info in the same Dynamo DB table.
- Use aws region us-west-2 for all these tasks.
- Write all the above research on SageMaker to a file named sagemaker_research.md in markdown format.
"""

#create the client for pricing MCP server
aws_price_client = create_mcp_client("uvx", ["awslabs.aws-pricing-mcp-server@latest"])

#create the client for Dynamo DB MCP server
ddb_client = create_mcp_client("uvx", ["awslabs.dynamodb-mcp-server@latest"])

model = BedrockModel(model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", temperature=0.1)

aws_price_agent = None
response = ""
with aws_price_client, aws_doc_client, ddb_client: # Specify multiple MCP clients
    # Get the tools from the MCP servers and concat them
    price_tools = aws_price_client.list_tools_sync()
    doc_tools = aws_doc_client.list_tools_sync()
    ddb_tools = ddb_client.list_tools_sync()
    all_tools = price_tools + doc_tools + ddb_tools + [file_write]

    #create the agent and pass the tools recieved from the MCP servers
    sagemaker_research_agent = Agent(model=model, tools=[all_tools])

    response = sagemaker_research_agent(task)

<div class="alert alert-info">
    <h4>Learn about all the MCP Servers from <a href="https://awslabs.github.io/mcp/">AWS Labs here.</a> </h4>
</div>


### 5. Let's Create a Basic MCP Server
##### Step1. Create a Python script with functions, using FastMCP

In [ ]:
%%writefile my_mcp_server.py 

from fastmcp import FastMCP
import asyncio

#create MCP server
mcp = FastMCP("My First MCP Server") # Creates an MCP server that can host your tools

@mcp.tool() #  Decorator that turns your Python functions into MCP tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

@mcp.tool()
def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@mcp.tool()
def divide(a: int, b: int) -> float:
    """Divide two numbers."""
    return a / b

if __name__ == "__main__":    
    mcp.run()

##### Step2. Create an agent that uses the above MCP server

In [ ]:
custom_mcp_client = create_mcp_client("python3", ["my_mcp_server.py"]) #Create an MCP client that connects to the MCP server

# specify the model
# model = BedrockModel(model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0", temperature=0.1)
model = BedrockModel(model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", temperature=0.1)

custom_mcp_agent = None
with custom_mcp_client:
    # Get the tools that the mcp client has to offer
    custom_mcp_tools = custom_mcp_client.list_tools_sync()

    #create the agent and pass the tools recieved from the MCP client
    custom_mcp_agent = Agent(model=model, tools=[custom_mcp_tools])

    response = custom_mcp_agent(f"What is (10*5-7+20)/9?")

### 5.Let's Build an MCP server with more meaningful functions
*Our MCP server will help 1/ query a Bedrock Knowledge Base, 2/ Rerank the query results from Knowledge Base.*<br>
*Amazon Bedrock Knowledge Base is a fully managed service that is useful in retrieval augmented generation (RAG) applications.*<br>
*It automatically handles data ingestion, chunking, embedding generation, and vector storage, enabling AI applications to provide accurate responses grounded in your organization's specific information.*<br>
*Once the chunks of data are retrueved from Bedrock Knowledge Base, you can further improve accuracy by re-ranking those chunks and pick the top ranked chunks to be sent to an LLM.*<br>

### Assumption: Bedrock Knowledge Base has already been Created.
Please store your Knowledge Base ID and the AWS region in kb.txt file.
![Document Igestion](doc_ingestion.png)

## Create MCP Server to retrieve and re-rank content from Bedrock Knowledge Base
![Document Retrieval Reranking](retrieve_re-rank.png)

#### Create a Python script with FastMCP decorators

In [ ]:
%%writefile kb_mcp_server.py
from fastmcp import FastMCP
import asyncio

from typing import Dict, List, Any, Optional, Union, Tuple
import boto3, json

# create MCP server
kb_mcp = FastMCP("My KB MCP Server") # Creates an MCP server that can host your tools

@kb_mcp.tool
def retrieve_from_kb(query: str, knowledge_base_id: str, max_results: int = 10, region_name = "us-west-2") -> List[str]:
    """
    Retrieves query search results from Bedrock Knowledge Base

    Args:
        query (str): The input query
        knowledge_base_id (str): The ID of the knowledge base
        num_results (int): Number of results to retrieve, defaults to 10
        region_name (str): AWS region, defaults to us-west-2
        
    Returns:
        List: Returns List of str
    """
    
    client = boto3.client("bedrock-agent-runtime", region_name=region_name)
    
    kb_response = client.retrieve(
        knowledgeBaseId=knowledge_base_id,
        retrievalQuery={
            "text": query
        },
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "numberOfResults": max_results,
                "overrideSearchType": "HYBRID"  # SEMANTIC, HYBRID
            }
        }
    )
    # Extract documents and metadata
    documents = []
    original_results = []
    
    for i, result in enumerate(kb_response.get("retrievalResults", [])):
        # Extract text from result
        text = ""
        if "content" in result and "text" in result["content"]:
            content_text = result["content"]["text"]
            if isinstance(content_text, list):
                text = " ".join([item.get("span", "") if isinstance(item, dict) else str(item) 
                                for item in content_text])
            else:
                text = str(content_text)
            
        # Store original result with metadata
        original_results.append({
            "position": i + 1,
            "score": result.get("scoreValue", 0),
            "text": text
        })
        documents.append(text)
    
    return documents

@kb_mcp.tool
def rerank_results(
    query,
    documents,
    rerank_model_arn = "arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0",
    reranked_result_count=5,
    region_name="us-west-2"
):
    """
    Rerank search results using a Bedrock reranking model.
    
    Args:
        query (str): The original query
        documents (list): List of document texts to rerank
        rerank_model_arn (str): ARN of the reranking model
        reranked_result_count (int): Number of reranked results to return, defaults to 5
        region_name (str): AWS region name, defaults to us-west-2
        
    Returns:
        dict: Dictionary containing original and reranked results
    """
    import boto3
    
    bedrock_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
    
    try:
        # Invoke the rerank API
        reranked = bedrock_client.rerank(
            queries=[{"textQuery": {"text": query}, "type": "TEXT"}],
            rerankingConfiguration={
                "bedrockRerankingConfiguration": {
                    "modelConfiguration": {"modelArn": rerank_model_arn},
                    "numberOfResults": reranked_result_count
                },
                "type": "BEDROCK_RERANKING_MODEL"
            },
            sources=[{
                "inlineDocumentSource": {"textDocument": {"text": doc}, "type": "TEXT"},
                "type": "INLINE"
            } for doc in documents]
        )
        
        # Process reranked results
        reranked_results = []
        for result in reranked.get("results", []):
            idx = result.get("index", 0)
            if 0 <= idx < len(documents):
                reranked_results.append({
                    "original_position": idx + 1,
                    "new_position": len(reranked_results) + 1,
                    "relevance_score": result.get("relevanceScore", 0),  # Full precision score
                    "text": documents[idx]
                })
        
        return {"original_results": documents, "reranked_results": reranked_results}
    
    except Exception as e:
        print(f"Reranking failed: {str(e)}")
        return {"original_results": documents, "reranked_results": []}

if __name__ == "__main__":
    # mcp.run(transport="streamable-http")
    kb_mcp.run()

#### Let's use the above MCP Server we created
We stored the Bedrock Knowledge Base ID and th AWS region in a file named kb.txt, which the Agent wille xtract and use.<br>
The Agent will use the MCP server to get the retirve and re-rank tools and pass them to the Agent.<br> 
For the input query, the Agent will query Bedrock Knowledge Base for relevant chunks, re-rank and send the chunks to the Agent.<br>
This approach of retrieve and rerank using embeddings stored in Bedrock Knwoledge Base is a much better approach than the one we used before to search AWS Documentation because that MCP server does not use vector embeddings.


In [ ]:
system_prompt = f"""
Extract Bedrock Knowledge base ID and AWS region from kb.txt file for querying Bedrock Knowledge Base.
Use the information to query Bedrock Knwoeldgebase and rerank.
"""

task = f"""
- What is Bedrock AgentCore Memory, Runtime, and Gateway? 
- How are they relevant to a healthcare customer?
- Now explain to me as if I am a 15-year old.
"""

# Create the MCP client
kb_custom_mcp_client = create_mcp_client("python3", ["kb_mcp_server.py"])

kb_custom_mcp_agent = None
response = ""
with kb_custom_mcp_client:
    # Get the tools that the mcp client has to offer
    kb_custom_mcp_tools = kb_custom_mcp_client.list_tools_sync()
    all_tools = kb_custom_mcp_tools + [file_read]

    #create the agent and pass the tools recieved from the MCP client
    custom_mcp_agent = Agent(model=model, tools=all_tools, system_prompt=system_prompt)

    response = custom_mcp_agent(task)